IMAGE SEGMENTATION USING TRADITIONAL MECHINE LEARNING

importing the necessary libraries

In [26]:
import numpy as np
import cv2
import pandas as pd
from sklearn import metrics
import pickle
from matplotlib import pyplot as plt

adding the image

In [7]:
img = cv2.imread("/content/test_img.tif")
#print(img)

changing the input image to a grey scale image 

In [8]:
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)


creating the empty data frame and entering the original image pixels

In [9]:
img2 = img.reshape(-1)
df = pd.DataFrame()
df['Original Image'] = img2


generating the list of gabor filter that to be applied on the image

In [10]:
num = 1  
kernels = []
for theta in range(2):   #Define number of thetas
    theta = theta / 4. * np.pi
    for sigma in (1, 3): 
        for lamda in np.arange(0, np.pi, np.pi / 4):  
            for gamma in (0.05, 0.5):   
            
                
                gabor_label = 'Gabor' + str(num)
                ksize=9
                kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)    
                kernels.append(kernel)
                fimg = cv2.filter2D(img2, cv2.CV_8UC3, kernel)
                filtered_img = fimg.reshape(-1)
                df[gabor_label] = filtered_img  #Labels columns as Gabor1, Gabor2, etc.
                print(gabor_label, ': theta=', theta, ': sigma=', sigma, ': lamda=', lamda, ': gamma=', gamma)
                num += 1

Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.0 : gamma= 0.05
Gabor2 : theta= 0.0 : sigma= 1 : lamda= 0.0 : gamma= 0.5
Gabor3 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.05
Gabor4 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor5 : theta= 0.0 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.05
Gabor6 : theta= 0.0 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.5
Gabor7 : theta= 0.0 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.05
Gabor8 : theta= 0.0 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.5
Gabor9 : theta= 0.0 : sigma= 3 : lamda= 0.0 : gamma= 0.05
Gabor10 : theta= 0.0 : sigma= 3 : lamda= 0.0 : gamma= 0.5
Gabor11 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.05
Gabor12 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor13 : theta= 0.0 : sigma= 3 : lamda= 1.5707963267948966 : gamma= 0.05
Gabor14 : theta= 0.0 : sigma= 3 : lamda= 1.5707963267948966 : gamma= 0.5
Gabor15 : theta= 0.0 : sigma= 3 : lamda= 2

ADDING OTHER FILTERS ON THE IMAGE TO COMPARE THE RESULT

Canny edge filter

In [11]:
edges = cv2.Canny(img, 100,200)
edges1 = edges.reshape(-1)
df['Canny Edge'] = edges1


importing the other filters from the sklearn library

In [12]:
from skimage.filters import roberts, sobel, scharr, prewitt

Roberts edge filter

In [13]:
edge_roberts = roberts(img)
edge_roberts1 = edge_roberts.reshape(-1)
df['Roberts'] = edge_roberts1

Sobels edge filter

In [14]:
edge_sobel = sobel(img)
edge_sobel1 = edge_sobel.reshape(-1)
df['Sobel'] = edge_sobel1

Scharr edge filter

In [15]:
edge_scharr = scharr(img)
edge_scharr1 = edge_scharr.reshape(-1)
df['Scharr'] = edge_scharr1

Prewitt edge filter

In [16]:
edge_prewitt = prewitt(img)
edge_prewitt1 = edge_prewitt.reshape(-1)
df['Prewitt'] = edge_prewitt1

GAUSSIAN FILTER with sigma=3

In [17]:
from scipy import ndimage as nd
gaussian_img = nd.gaussian_filter(img, sigma=3)
gaussian_img1 = gaussian_img.reshape(-1)
df['Gaussian s3'] = gaussian_img1

GAUSSIAN FILTER with sigma=7

In [18]:
gaussian_img2 = nd.gaussian_filter(img, sigma=7)
gaussian_img3 = gaussian_img2.reshape(-1)
df['Gaussian s7'] = gaussian_img3

MEDIAN with sigma=3


In [19]:
#MEDIAN with sigma=3
median_img = nd.median_filter(img, size=3)
median_img1 = median_img.reshape(-1)
df['Median s3'] = median_img1


VARIANCE with size=3

In [20]:
variance_img = nd.generic_filter(img, np.var, size=3)
variance_img1 = variance_img.reshape(-1)
df['Variance s3'] = variance_img1

add a column in the data frame for the Labels

In [21]:
labeled_img = cv2.imread('/content/test_img.tif')
labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)
labeled_img1 = labeled_img.reshape(-1)
df['Labels'] = labeled_img1
print(df.head())
#df.to_csv("Gabor.csv")

   Original Image  Gabor1  Gabor2  ...  Median s3  Variance s3  Labels
0               0       0       0  ...          0            0       0
1               0       0       0  ...          0            0       0
2               0       0       0  ...          0            0       0
3               0       0       0  ...          0            0       0
4               0       0       0  ...          0            0       0

[5 rows x 43 columns]


preparing the data into training and test sets

In [22]:
Y = df["Labels"].values
X = df.drop(labels = ["Labels"], axis=1) 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=20)


importing the RandomForestClassifier model and fitting the data

In [23]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 10, random_state = 42)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

to check the prediction

In [24]:
prediction_test_train = model.predict(X_train)
prediction_test = model.predict(X_test)

print ("Accuracy on training data = ", metrics.accuracy_score(y_train, prediction_test_train))
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction_test))


Accuracy on training data =  0.9991093927202251
Accuracy =  0.8469980047161255


importance of each column for segmentation

In [25]:
feature_list = list(X.columns)
feature_imp = pd.Series(model.feature_importances_,index=feature_list).sort_values(ascending=False)
#print(feature_imp)

Storing the model

In [ ]:

filename = "sandstone_model"
pickle.dump(model, open(filename, 'wb'))

Adding the input image to the model to observe the segmented image


In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(X)
segmented = result.reshape((img.shape))


Visualization of the image

In [ ]:

plt.imshow(segmented, cmap ='jet')
plt.imsave('segmented_rock_RF_100_estim.jpg', segmented, cmap ='jet')